In [104]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from keras.models import load_model
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [38]:
def load_image(path):
    img = cv2.imread(path)
    return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

def show_image(img, ):
    plt.imshow(img, 'gray')
    plt.show()

def image_bin(image_gs):
    height, width = image_gs.shape[0:2]
    ret, image_bin = cv2.threshold(image_gs, 127, 255, cv2.THRESH_BINARY)
    return image_bin

alphabet = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'R', 'S', 'T', 'U', 'V', 'X', 'Z']

In [4]:
plates = []
folder_path = '../datasets/train/'

for img_name in os.listdir(folder_path):
    img_path = os.path.join(folder_path, img_name)
    img = load_image(img_path)
    if'plate' in img_name:
        plates.append(img)


In [24]:
folder_path = '../datasets/font/train/'
for letter in alphabet:
    folder = os.path.join(folder_path, letter)
    if not os.path.exists(folder):
        os.makedirs(folder)

In [95]:
def erode(image, kernel_shape=(3, 3), iter=1):
    kernel = np.ones(kernel_shape)
    return cv2.erode(image, kernel, iterations=iter)

def resize_region(region, shape=(28, 28)):
    return cv2.resize(region, shape, interpolation=cv2.INTER_NEAREST)

def select_roi_test(image_orig, image_bin):
    image_bin2 = erode(image_bin, (4, 1), 1)        # da bih spojio kvacice
    contours, hierarchy = cv2.findContours(image_bin2.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    sorted_regions = []
    regions_array = []
    y_sub = 4
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        area = cv2.contourArea(contour)
        if area > 150 and h < 100 and h > 20 and w > 5 and w < 100:
            y +=y_sub
            h -= 2 * y_sub
            region = image_bin[y:y+h+1, x:x+w+1]
            regions_array.append([resize_region(region), (x, y, w, h)])
            cv2.rectangle(image_orig, (x, y), (x + w, y + h), (0, 255, 0), 2)
    
    regions_array = sorted(regions_array, key=lambda x: x[1][0])
    sorted_regions = [region[0] for region in regions_array]
    return image_orig, sorted_regions

def winner(output):
    return max(enumerate(output), key=lambda x: x[1])[0]

def display_result(outputs, alphabet):
    result = []
    for output in outputs:
        result.append(alphabet[winner(output)])
    return result

def select_roi_train(image_orig, image_bin):
    image_bin2 = erode(image_bin, (3, 1), 2)        # da bih spojio kvacice
    contours, hierarchy = cv2.findContours(image_bin2.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    sorted_regions = []
    regions_array = []
    y_sub = 4
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        area = cv2.contourArea(contour)
        if area > 700 and h < 100 and h > 40 and w > 10:
            y +=y_sub
            h -= 2 * y_sub
            region = image_bin[y:y+h+1, x:x+w+1]
            regions_array.append([resize_region(region), (x, y, w, h)])
            cv2.rectangle(image_orig, (x, y), (x + w, y + h), (0, 255, 0), 2)
    
    regions_array = sorted(regions_array, key=lambda x: x[1][0])
    sorted_regions = [region[0] for region in regions_array]
    return image_orig, sorted_regions

In [70]:
cnn = load_model('../model.h5')
folder_path = '../datasets/font/train/'
 
for img in plates:
    img_bin = image_bin(img)
    selected_img, sorted_regions = select_roi_test(img.copy(), img_bin)
    if sorted_regions:      
        result = cnn.predict(np.array(sorted_regions, np.float32))
        result = display_result(result, alphabet)

        for i, region in enumerate(sorted_regions):
            folder = os.path.join(folder_path, result[i])
            img_names = os.listdir(folder)
            j = max(img_names, default=0)
            j = str(j)
            j = j.rstrip('.jpg')
            j = int(j) + 1
            folder = os.path.join(folder, str(j))
            cv2.imwrite(folder + ".jpg", region)


1/1 [==============================] - 0s 64ms/step


In [132]:
datagen = ImageDataGenerator(
    rotation_range=6,
    height_shift_range=0.05
)

img = load_image('../datasets/font/plate_font.png')
img_bin = image_bin(img)
selected_img, sorted_regions = select_roi_train(img.copy(), img_bin)

alphabet_images = []
for region in sorted_regions:
    region = keras.preprocessing.image.img_to_array(region)
    alphabet_images.append(region)

alphabet_images = np.array(alphabet_images)
alphabet_images = np.expand_dims(alphabet_images, axis=0)
batch_size = alphabet_images.shape[1]

alphabet_images = np.reshape(alphabet_images, (batch_size,) + alphabet_images.shape[2:])
img_iter = datagen.flow(alphabet_images, batch_size=batch_size)

In [136]:
batch = img_iter.next()
result = cnn.predict(np.array(batch, np.float32))
result = display_result(result, alphabet)

for i, img in enumerate(batch):
    folder = os.path.join(folder_path, result[i])
    if os.path.exists(folder):                          # necemo imati foldere sa slovima ČĆŽĐŠWY
        img_names = os.listdir(folder)
        j = max(img_names, default=0)
        j = str(j)
        j = j.rstrip('.jpg')
        j = int(j) + 1
        folder = os.path.join(folder, str(j))
        cv2.imwrite(folder + ".jpg", img)


2/2 [==============================] - 0s 8ms/step
